# **Universal Bank Loan Analysis using Super Machine Learner model**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import zipfile 

In [188]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
zip = zipfile.ZipFile('archive.zip')
zip.extractall()

In [ ]:
def Loan(df):
  for i in range(len(df)):
    if df['Mortgage'][i]!=0 and (df['Online'][i]==0 or df['CreditCard'][i]==0 or df['CD Account'][i]==0 or df['Securities Account'][i]==0):
       df['Loan/Debt'][i]=1
    else:
       df['Loan/Debt'][i]=0
  return df

In [ ]:
df=pd.read_csv("UniversalBank.csv")
df['CCAvg']=round(df['CCAvg']).astype(int)
df=df.drop(columns=['ID','ZIP Code'])
df['Loan/Debt']=0
df=Loan(df)
df=df.dropna(how='any')
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Loan/Debt
0,25,1,49,4,2,1,0,0,1,0,0,0,0
1,45,19,34,3,2,1,0,0,1,0,0,0,0
2,39,15,11,1,1,1,0,0,0,0,0,0,0
3,35,9,100,1,3,2,0,0,0,0,0,0,0
4,35,8,45,4,1,2,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,2,3,0,0,0,0,1,0,0
4996,30,4,15,4,0,1,85,0,0,0,1,0,1
4997,63,39,24,2,0,3,0,0,0,0,0,0,0
4998,65,40,49,3,0,2,0,0,0,0,1,0,0


In [102]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [103]:
y=y.reshape(len(y),1)

In [197]:
def get_models():
  models=list()
  models.append(LogisticRegression(solver='liblinear'))
  models.append(DecisionTreeClassifier())
  models.append(SVC(gamma='scale', probability=True))
  models.append(GaussianNB())
  models.append(KNeighborsClassifier())
  models.append(AdaBoostClassifier())
  models.append(BaggingClassifier(n_estimators=10))
  models.append(RandomForestClassifier(n_estimators=10))
  models.append(ExtraTreesClassifier(n_estimators=10))
  models.append(MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, learning_rate='adaptive', max_iter=500))
  return models

In [186]:
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	# define split of data
	kfold = KFold(n_splits=10, shuffle=True)
	# enumerate splits
	for train_ix, test_ix in kfold.split(X):
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			# store columns
			fold_yhats.append(yhat)
		# store fold yhats as columns
		meta_X.append(np.hstack(fold_yhats))
	return np.vstack(meta_X), np.asarray(meta_y)

In [144]:
def base_models(X_train,y_train,models):
  print("Models initialised")
  for i in range(len(models)):
    models[i].fit(X_train,y_train)
    print("{} model training completed".format(str(models[i])))
  print("models construction completed")
  


In [122]:
def fit_meta_model(X_train, y_train):
	model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, learning_rate='adaptive', max_iter=500)
	model.fit(X_train, y_train)
	return model

In [123]:
def evaluate_models(X_test, y_test, models):
	for model in models:
		yhat = model.predict(X_test)
		acc = accuracy_score(y_test, yhat)
		print('%s: %.3f' % (model.__class__.__name__, acc*100))

In [124]:
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X)
		meta_X.append(yhat)
	meta_X = np.hstack(meta_X)
	# predict
	return meta_model.predict(meta_X)

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [126]:
print('Train', X_train.shape, y_train.shape, 'Test', X_test.shape, y_test.shape)

Train (3750, 12) (3750, 1) Test (1250, 12) (1250, 1)


In [198]:
models = get_models()

In [199]:
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

In [200]:
print('Meta ', meta_X.shape, meta_y.shape)

Meta  (5000, 20) (5000, 1)


In [201]:
base_models(X_train, y_train, models)

Models initialised
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False) model training completed
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') model training completed
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probabil

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None) model training completed
BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False) model training completed
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) mode

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False) model training completed
models construction completed


In [202]:
meta_model = fit_meta_model(meta_X, meta_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [203]:
evaluate_models(X_test, y_test, models)

LogisticRegression: 98.960
DecisionTreeClassifier: 100.000
SVC: 99.280
GaussianNB: 99.280
KNeighborsClassifier: 99.280
AdaBoostClassifier: 100.000
BaggingClassifier: 100.000
RandomForestClassifier: 99.920
ExtraTreesClassifier: 99.920
MLPClassifier: 99.760


In [204]:
yhat = super_learner_predictions(X_test, models, meta_model)
print('Super Learner: %.3f' % (accuracy_score(y_test, yhat) * 100))

Super Learner: 100.000
